In [1]:
import threading
from pathlib import Path

import lib_ipmu_daq_api as api
import lib_ipmu_daq_config as config
import lib_ipmu_daq_generator as gen
import lib_ipmu_daq_process as post
import lib_ipmu_daq_graph as graph

%load_ext autoreload
%autoreload 2

In [2]:
preset = config._loadConfig(Path("./_config_preset.toml"))
run = config._loadConfig(Path("./_config_run.toml"))
app_config = config.initParams(preset, run, debug=True)


[Io]
  sample_rate         : 100000
  gen_chunk_sec       : 0.100000
  proc_interval       : 0.125000
  queue_depth         : 40
  quad_depth          : 40

[Gui]
  display_sec         : 3600
  plot_sec            : 0.150000
  gui_interval_ms     : 50
  pruning             : 100

[Logging]
  log_chunk           : 1024
  log_data_num        : 4

[Encoder Postproc]
  quadpulse_width     : 0.000250
  threshold           : 2.500000

[Debug Encoder]
  input_velocity      : 0.100000
  pulse_height        : 5.000000
  pulse_duty          : 0.500000
  pulse_phase_A       : 0.000000

[Debug Power]
  amplitude           : 0.300000
  phase               : 0.000000

[Driver]
  t_DC                : 10
  pps                 : 0
  step                : 100
  rst                 : 1
  target_freq         : 1

[Dependent]
  chunk_sec           : 0.100000
  n_samples_gen       : 10000
  samples_proc        : 12500
  history             : 150
  count_history       : 80
  velo_history        : 80
  pow_

In [3]:
try:
    app = api.DAQApp(config=app_config, DEBUG=True)
    app.setup() # initialize widgets
    app.initStorer(runs_dir=config.RUNS_DIR) # initialize HDF5 storage
    app.initLogger()  # Initialize logger in the main thread to avoid issues

    # generater for debug
    generator = gen.Generator(config=app_config, buf_q=app.buf_q, stop_event=app.stop_event)

    # processor
    processor = post.Processor(config=app_config, buf_q=app.buf_q, quad_q=app.quad_q, stop_event=app.stop_event, h5f=app.h5f, dset=app.dset, debug=app.DEBUG, logger=app.logger)

    # Graph
    gui = graph.DAQGUI(config=app_config, quad_q=app.quad_q, stop_event=app.stop_event)

    # 5. Create the thread objects individually
    #logger_thread = threading.Thread(target=app.initLogger, daemon=True)
    generator_thread = threading.Thread(target=generator.run, daemon=True)
    processor_thread = threading.Thread(target=processor.run, daemon=True)

    # 6. Start the threads (now controlled from the top level)
    #logger_thread.start()
    generator_thread.start()
    processor_thread.start()

    gui.run()

except KeyboardInterrupt:
    print("KeyboardInterrupt received. Shutting down...")
    app.shutdown()
finally:
    app.shutdown()
    #logger_thread.join(timeout=0.1)
    generator_thread.join(timeout=0.1)
    processor_thread.join(timeout=0.1)



Shared resources (Queues, Stop Event) are set up.
DEBUG mode is ON.
HDF5 dataset created at: C:\Users\USER\Desktop\ipmu_DAQ\src\..\runs\251006142951.h5
EPOCH = 212668.930168, wall = 1030.93 ms, jitter = 1030.93 ms  delta c=+25, v= 0.098, v_ref= 0.000, time_p = 0.062495, P_tot = 0.415789
EPOCH = 212669.159596, wall = 229.43 ms, jitter = 229.43 ms  delta c=+26, v= 0.102, v_ref= 0.000, time_p = 0.187495, P_tot = 0.415782
EPOCH = 212669.348307, wall = 188.71 ms, jitter = 188.71 ms  delta c=+25, v= 0.098, v_ref= 0.000, time_p = 0.312495, P_tot = 0.415719
EPOCH = 212669.534095, wall = 185.79 ms, jitter = 185.79 ms  delta c=+26, v= 0.102, v_ref= 0.000, time_p = 0.437495, P_tot = 0.415731
EPOCH = 212669.687666, wall = 153.57 ms, jitter = 153.57 ms  delta c=+25, v= 0.098, v_ref= 0.000, time_p = 0.562495, P_tot = 0.415690
EPOCH = 212670.884719, wall = 1197.05 ms, jitter = 1197.05 ms  delta c=+26, v= 0.102, v_ref= 0.000, time_p = 0.687495, P_tot = 0.415779
EPOCH = 212671.062993, wall = 178.27 ms,